<div style="position: relative;">
<img src="https://user-images.githubusercontent.com/7065401/98728503-5ab82f80-2378-11eb-9c79-adeb308fc647.png"></img>

<h1 style="color: white; position: absolute; top:27%; left:10%;">
     Advanced Python
</h1>
<h2 style="color: white; position: absolute; top:36%; left:10%;">
    Iterators, Generators, Context Managers, and Decorators
</h2>


<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:58%; left:10%;">
    David Mertz, Ph.D.
</h3>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:63%; left:10%;">
    Data Scientist
</h3>
</div>

# Decorators

A decorator, at base, is nothing more than syntax sugar for a callable that takes one function or class as an argument, and returns ... **something**.  Given how deeply introspective Python can be, you can modify functions and classes in pretty much any conceivable way (some take more work than others).  In general, decorators are simply a convenient way of expressing a kind of modification that you will potentially want to apply to many functions or classes.

Hopefully without belaboring the matter too much, what a decorator returns really can be *anything*.  Most of the time it is a somewhat modified class or function that performs a largely similar function to the undecorated version.  But the syntax is not so constrained.

In [1]:
def func_to_num(fn):
    return 42

@func_to_num
def fibonacci(max=float('inf'), a=1, b=2):
    while a < max:
        yield a
        a, b = b, a+b
        
print(fibonacci)

42


In [2]:
try:
    fibonacci(999)
except Exception as err:
    print(err)

'int' object is not callable


The decorator `@func_to_num` is a generally terrible decorator with no reasonable purpose.  But it **is** a decorator.

## Transforming Functions

Decorators can be a powerful way of expressing "cross-cutting" behavior that you want to apply to different functions. A very simple, but still useless, decorator is the identity decorator.  It simply returns a function that behaves the same way as the function passed into it.  However, the way we write this shows the structure of writing more useful ones.

In [3]:
def decorate(func):
    def new_func(arg):
        return func(arg)
    return new_func

Even without decorators, We could modify the function using `decorate()` and rebind it to the same name.

In [4]:
def fn1(x):
    return x + 1

print("Original fn1() answer:", fn1(5))

fn1 = decorate(fn1)
print("Modified fn1() answer:", fn1(5))

Original fn1() answer: 6
Modified fn1() answer: 6


It is generally prettier to do a semantically identical thing using a decorator like the following.  Decorators are higher-order functions that modify a function, and then rebind the new function the same name.  Their main advantage is simply that they come at the start of a function definition rather at the end of a long function definition (or elsewhere than that even).

In [5]:
def fn2(x):
    return x + 1

@decorate
def fn3(x):
    return x + 1

The function returned by `@decorate` is bound to the same name `fn3` and replaces the original, decorated function.  In the example above, ```new_func``` becomes the new implementation of ```fn3```. Let's contrast `fn2()` with `fn3()`.

In [6]:
print(f"Undecorated: {fn2.__name__}(17) == {fn2(17)}")
print(f"Decorated: {fn3.__name__}(17) == {fn3(17)}")

Undecorated: fn2(17) == 18
Decorated: new_func(17) == 18


### A first useful decorator

Let's imagine that we want to allow `fun()` to operate on sequences of numbers, but we only want to define it as an operation on a single number.

In [7]:
def map_scalar(func):
    def map_to_seq(*args):
        return list(map(func, args))
    return map_to_seq

In [8]:
@map_scalar
def add_one(x):
    return x + 1

In [9]:
add_one(3)

[4]

In [10]:
add_one(10, 20, 30, 40)

[11, 21, 31, 41]

## Change behavior on duck-type of arguments

A slightly more interesting variant on the above is to turn Python functions into something like NumPy ufuncs.  That is, perhaps we would like them to operate on either scalars or sequences (preserving the sequence type).

In [11]:
def elementwise(fn):
    "Transform a function on scalars into a function on collections"
    
    def newfn(arg):
        "Inner function of elementwise decorator"
        # Treat a string as scalar even though it is iterable
        if isinstance(arg, str):
            return fn(arg)        
        try:                      # Something iterable
            return type(arg)(map(fn, arg))
        except TypeError as err:  # Assume it is scalar
            return fn(arg)
        
    return newfn

In [12]:
@elementwise
def compute(x):
    "Calculate one less than the cube of an input"
    return x**3 - 1

In [13]:
compute(5)

124

In [14]:
compute([1, 2+3j, 3.14]) 

[0, (-47+9j), 29.959144000000002]

In [15]:
compute({8, 9, 10})

{511, 728, 999}

# Where naïve wrapping goes wrong

The simple examples above show a general pattern for creating decorators.  However, they are fragile in the face of object introspection, whether for debugging or other purposes.  Suppose we try to use a function and it goes wrong:

In [16]:
try: 
    compute("Ionesco")
except Exception as err:
    print(repr(err))

TypeError("unsupported operand type(s) for ** or pow(): 'str' and 'int'")


That is not so terrible as an error message, we should try to figure out how we are meant to use the function.

In [17]:
help(compute)

Help on function newfn in module __main__:

newfn(arg)
    Inner function of elementwise decorator



We provided a docstring for `compute()`, but it got lost when it was decorated!  What we actually have is an instance of the inner function created within the decorator; that one does not have any docstring.  The actual function object only has the generic description of what it does within the decorator.

## Using `functools.wraps`

As we do further introspection of `compute` we become more troubled that it is *not* really the function we created.  Fortunately, the solution here is very simple.  We simply need to use `functools.wraps` to cleanup these details for us.

In [18]:
import functools

def elementwise(fn):
    @functools.wraps(fn)  # <-- Add this to the interior function
    def newfn(arg):
        if isinstance(arg, str):  # string as scalar
            return fn(arg)        
        try:                      # Something iterable
            return type(arg)(map(fn, arg))
        except TypeError as err:  # Assume it is scalar
            return fn(arg)
    return newfn

In [19]:
@elementwise
def compute(x):
    "Calculate one less than the cube of input value(s)"
    return x**3 - 1

compute(5), compute((1, 2, 3))

(124, (0, 7, 26))

In [20]:
help(compute)

Help on function compute in module __main__:

compute(x)
    Calculate one less than the cube of input value(s)



In [21]:
compute.__name__

'compute'

## Combining decorators

It is often possible and useful to combine several decorators.  For example, let us make a decorator for logging operations performed.

In [22]:
from datetime import datetime
def log_calls(fn):
    # Name logfile after the function name
    logfile = open(f"{fn.__name__}.log", 'w')
    @functools.wraps(fn)
    def inner(*args, **kws):
        # Inner function accepts arbitrary positional and named args
        result = fn(*args, **kws)
        # Perform the logging
        print(datetime.now().isoformat(), file=logfile, end=" ")
        # To simplify, not logging keyword args in this example
        args = map(str, args)
        print(f"{fn.__name__}({','.join(args)}) → {result}", 
              file=logfile, flush=True)
        return result
    return inner

In [23]:
@log_calls
@elementwise
def compute(x):
    "Calculate one less than the cube of input value(s)"
    return x**3 - 1

In [24]:
from time import sleep
for data in [5, (1, 2, 3), [1, 2+3j, 3.14], {8, 9, 10}]:
    compute(data)
    sleep(1.2)

In [25]:
!cat compute.log

2021-03-03T23:42:49.478722 compute(5) → 124
2021-03-03T23:42:50.681899 compute((1, 2, 3)) → (0, 7, 26)
2021-03-03T23:42:51.882303 compute([1, (2+3j), 3.14]) → [0, (-47+9j), 29.959144000000002]
2021-03-03T23:42:53.083855 compute({8, 9, 10}) → {728, 999, 511}
